In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pprint import pprint
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.207:7077") \
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .appName("pa_test")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [ ]:
df = spark_session.read.json("hdfs://192.168.2.207:9000/user/ubuntu/sample_data.json").cache()\

In [4]:
lines = spark_context.textFile("hdfs://192.168.2.207:9000/user/ubuntu/negative-words.txt")
negative_words = lines.collect()
batchSize = 400

10000


In [19]:
reddit_body_df = df.select("body")
import pyspark.sql.functions as psf
count = 0

for index in range(0,len(negative_words),batchSize):
    if index == 0:
        filtered_df = reddit_body_df.filter(psf.col('body').rlike('(^|\s)(' + '|'.join(negative_words[index:batchSize]) + ')(\s|$)'))
    else:
        df_temp = reddit_body_df.filter(psf.col('body').rlike('(^|\s)(' + '|'.join(negative_words[index:batchSize]) + ')(\s|$)'))
        filtered_df = filtered_df.union(df_temp)

unique_negative_df = filtered_df.dropDuplicates()
print('number of reddit bodies which have at least one negative word: ', unique_negative_df.count())

number of reddit bodies which have at least one negative word:  4129
